In [6]:
import os
import laspy
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm import tqdm

In [7]:
for_instance_folder = Path.home() / 'datasets/FORinstance'
nibio_mls_folder = Path.home() / 'datasets/NIBIO_MLS'
output_folder = Path.home() / 'datasets/MixedDataset'

for_instance_folder.mkdir(parents=True, exist_ok=True)
nibio_mls_folder.mkdir(parents=True, exist_ok=True)
output_folder.mkdir(parents=True, exist_ok=True)

classes = {
    'terrain': 0,
    # 'low_vegetation': 1,
    'stem': 1,
    'canopy': 2
}

In [8]:
def load_point_clouds(folder):
    files = sorted(
        [f for f in folder.rglob("*") if f.is_file() and f.suffix.lower() in ('.laz', '.las')],
        key=lambda f: f.name
    )

    for path in files:
        ext = path.suffix.lower()
        file = laspy.read(path)

        min_coords = np.array([file.x.min(), file.y.min(), file.z.min()], dtype=np.int64)
        mins_world = min_coords * file.header.scales + file.header.offsets
        file.header.offsets -= mins_world

        intensity = np.array(file.intensity)
        min_intensity = np.min(intensity)
        max_intensity = np.max(intensity)

        file.add_extra_dims([
            laspy.ExtraBytesParams(name="norm_intensity", type=np.float32),
            laspy.ExtraBytesParams(name="semantic_pred", type=np.int16),
            laspy.ExtraBytesParams(name="instance_pred", type=np.int16)
        ])

        file.norm_intensity = (intensity - min_intensity) / (max_intensity - min_intensity)
        yield ext, file


In [9]:
for_instance_classes = {
    'unclassified': 0,
    'low_vegetation': 1,
    'terrain': 2,
    'out_points': 3,
    'stem': 4,
    'live_branches': 5,
    'woody_branches': 6
}

for i, (ext, file) in enumerate(tqdm(load_point_clouds(for_instance_folder))):
    mask = (file.classification != for_instance_classes['out_points']) & (file.classification != for_instance_classes['unclassified'])  # Eliminamos puntos no clasificados o inválidos
    file.points = file.points[mask]

    semantic_labels = np.array(file.classification)
    remap = np.copy(semantic_labels)

    remap = np.where(semantic_labels == for_instance_classes['low_vegetation'], classes['terrain'], remap)
    remap = np.where(semantic_labels == for_instance_classes['terrain'], classes['terrain'], remap)
    remap = np.where(semantic_labels == for_instance_classes['stem'], classes['stem'], remap)
    remap = np.where(semantic_labels == for_instance_classes['live_branches'], classes['canopy'], remap)
    remap = np.where(semantic_labels == for_instance_classes['woody_branches'], classes['canopy'], remap)

    file.semantic_pred = remap
    file.instance_pred = file.treeID
    file.write(output_folder / f'plot_FORinstance_{i}{ext}')


28it [00:49,  1.78s/it]


In [10]:
nibio_mls_classes = {
    'ground': 1,
    'vegetation': 2,
    'lying_deadwood': 3,
    'stems': 4
}

for i, (ext, file) in enumerate(tqdm(load_point_clouds(nibio_mls_folder))):
    semantic_labels = np.array(file.label)
    remap = np.copy(semantic_labels)

    remap = np.where(semantic_labels == nibio_mls_classes['ground'], classes['terrain'], remap)
    remap = np.where((semantic_labels == nibio_mls_classes['vegetation']) & (file.treeID == 0), classes['terrain'], remap)
    remap = np.where((semantic_labels == nibio_mls_classes['vegetation']) & (file.treeID != 0), classes['canopy'], remap)
    remap = np.where(semantic_labels == 3, classes['terrain'], remap)
    remap = np.where(semantic_labels == 4, classes['stem'], remap)

    file.semantic_pred = remap
    file.instance_pred = file.treeID
    file.write(output_folder / f'plot_NIBIO_MLS_{i}{ext}')


64it [00:48,  1.33it/s]
